# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
city_data = pd.read_csv('../WeatherPy/city_data.csv',index_col=0)
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,banjar,-8.1900,114.9675,82.40,83,20,3.44,ID,1612587579
1,rikitea,-23.1203,-134.9692,78.15,71,52,13.94,PF,1612587580
2,ormara,25.2088,64.6357,68.18,38,0,2.89,PK,1612587580
3,itarema,-2.9248,-39.9167,78.98,78,46,11.14,BR,1612587571
4,yulara,-25.2406,130.9889,86.00,21,62,10.36,AU,1612587575
...,...,...,...,...,...,...,...,...,...
552,san quintin,30.4833,-115.9500,57.11,81,0,8.79,MX,1612587571
553,cruz grande,16.7333,-99.1333,71.60,84,30,2.91,MX,1612587793
554,dunedin,-45.8742,170.5036,60.01,78,14,23.00,NZ,1612587536
555,chumikan,54.7183,135.3150,-5.87,86,100,11.68,RU,1612587794


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

In [4]:
# Use the Lat and Lng as locations
locations = city_data[["Lat","Lng"]]
#locations
# Humidity as the weight
humidity = city_data["Humidity"].astype(float)
maxhumidity = humidity.max()
maxhumidity

100.0

In [6]:
# center is the center of the united states
fig = gmaps.figure(zoom_level=4, center=(39.82, -98.35))

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# a max temperature lower than 80 but higher than 70
best_city_df = city_data.loc[city_data["Max Temp"].between(70,80,inclusive=False)]
# wind speed less than 10
best_city_df = best_city_df.loc[city_data["Wind Speed"] < 9]
# Zero cloudiness
best_city_df = best_city_df.loc[(city_data["Cloudiness"] == 0), :]
# drop any null values
best_city_df = best_city_df.dropna(how='any')
best_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
163,tura,25.5198,90.2201,77.45,31,0,3.85,IN,1612587677
168,pisco,-13.7000,-76.2167,71.60,78,0,3.44,PE,1612587591
230,amanganj,24.4333,80.0333,72.21,30,0,8.95,IN,1612587699
251,umarkot,25.3616,69.7362,73.74,20,0,4.79,PK,1612587705
269,maceio,-9.6658,-35.7353,75.20,88,0,6.91,BR,1612587711
279,celestun,20.8667,-90.4000,74.03,73,0,7.72,MX,1612587714
319,san juan,-31.5375,-68.5364,77.77,35,0,4.03,AR,1612587726
345,yenagoa,4.9247,6.2642,77.29,91,0,2.04,NG,1612587734
346,moron,-34.6534,-58.6198,71.01,79,0,4.63,AR,1612587734
410,tucuman,-26.8241,-65.2226,73.40,83,0,3.44,AR,1612587746


In [24]:
# remove the index column, rest_index to renumber index
best_city_df.reset_index(inplace=True)
del best_city_df['index']
best_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tura,25.5198,90.2201,77.45,31,0,3.85,IN,1612587677
1,pisco,-13.7000,-76.2167,71.60,78,0,3.44,PE,1612587591
2,amanganj,24.4333,80.0333,72.21,30,0,8.95,IN,1612587699
3,umarkot,25.3616,69.7362,73.74,20,0,4.79,PK,1612587705
4,maceio,-9.6658,-35.7353,75.20,88,0,6.91,BR,1612587711
5,celestun,20.8667,-90.4000,74.03,73,0,7.72,MX,1612587714
6,san juan,-31.5375,-68.5364,77.77,35,0,4.03,AR,1612587726
7,yenagoa,4.9247,6.2642,77.29,91,0,2.04,NG,1612587734
8,moron,-34.6534,-58.6198,71.01,79,0,4.63,AR,1612587734
9,tucuman,-26.8241,-65.2226,73.40,83,0,3.44,AR,1612587746


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
# store info City, Country, Lat, Lng into variable name hotel_df
hotel_df = pd.DataFrame(best_city_df, columns=["City", "Country", "Lat", "Lng"])
# add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,tura,IN,25.5198,90.2201,
1,pisco,PE,-13.7000,-76.2167,
2,amanganj,IN,24.4333,80.0333,
3,umarkot,PK,25.3616,69.7362,
4,maceio,BR,-9.6658,-35.7353,
5,celestun,MX,20.8667,-90.4000,
6,san juan,AR,-31.5375,-68.5364,
7,yenagoa,NG,4.9247,6.2642,
8,moron,AR,-34.6534,-58.6198,
9,tucuman,AR,-26.8241,-65.2226,


In [35]:
# Set parameters to search for hotels with 5000 meters
hotels_list=[]
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']
    #set parameters dictionary
    params = {"location": f"{lat},{lng}",
             "radius": 5000,
             "types": "hotel",
             "key": g_key}
    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #make request
    hotel_name = requests.get(base_url, params=params)
    #convert to jason
    hotel_name = hotel_name.json()
    
    try:
        hotels_list.append(hotel_name['results'][0]['name'])
    except:
        hotels_list.append("")
hotel_df["Hotel Name"] = hotels_list
# drop any rows that may have na for hotel because there was not a match for hotel 5000 radius
hotel_df = hotel_df.dropna(how='any')
hotel_df          

,City,Country,Lat,Lng,Hotel Name
0,tura,IN,25.5198,90.2201,Tura
1,pisco,PE,-13.7000,-76.2167,Pisco
2,amanganj,IN,24.4333,80.0333,Amanganj
3,umarkot,PK,25.3616,69.7362,Umarkot
4,maceio,BR,-9.6658,-35.7353,Maceió
5,celestun,MX,20.8667,-90.4000,Celestún
6,san juan,AR,-31.5375,-68.5364,San Juan
7,yenagoa,NG,4.9247,6.2642,Yenagoa
8,moron,AR,-34.6534,-58.6198,Morón
9,tucuman,AR,-26.8241,-65.2226,San Miguel de Tucumán


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
marker_hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))